<a href="https://colab.research.google.com/github/Petrichoeur/TER/blob/master/MultiStream/process_list_for_json_with_txt_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd  
import numpy as np
# Changer data anae final par un autre enfant 

def Reverse(tuples): 
    new_tup = tuples[::-1] 
    return new_tup 

data_child = pd.read_csv('data_anae_final_.csv',
                           sep = '\t',
                           encoding = 'utf-8',
                           index_col=False)

# On supprime la colonne qui duplique les index
data_child = data_child.drop(columns = 'Unnamed: 0') 
data_child=data_child.sort_values(by = ['age','seconde_debut']) 

data_child.head()
data_child['ind']=[ i for i in range(len(data_child))]
data_child= data_child.set_index('ind')


In [0]:

pho_child = data_child.where(data_child['type']=='pho').dropna(subset=['type'])
#pho_child.to_csv('phono_anae.csv',sep=';') 



In [8]:
#data=pd.read_csv('phono_anae.csv',sep=';')
data=pho_child
pho=data[['age','contenu']] 

age=data['age'] 
l=[i for i in range(len(age))]
k=[age.values]
k.append(l)
tuples = list(zip(*k))
index = pd.MultiIndex.from_tuples(tuples, names=['index', 'age'])
data.set_index(index,inplace=True)

D={} 
pho=data[['contenu']]
pho['contenu']=pho['contenu'].apply(lambda x: str(x))
for i in list(zip(*pho.index))[0]:  
    z=pho['contenu'].loc[i].values
    y= ' '.join(z) 
    D[i]=y.split()
    
    
flemme = pd.DataFrame(list(zip(*pho.index))[0]) 

FLEMME=flemme[0].unique()

df = {} 
for i in FLEMME : 
    df[i]=pd.DataFrame(D[i], index= [i for _ in range(len(D[i]))] , columns=['mot']) 

    
df2=pd.concat([df[i] for i in FLEMME]) 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:

IPA = "a,r,l,e,s,i,ɛ,ə,t,k,p,d,m,ɑ̃,n,u,v,o,y,ɔ̃,ʒ,ɔ,ɛ̃,f,b,j,w,ɥ,z,ø,ʃ,œ̃,œ,g,ɑ,ɲ,tr,kr,dr,gr" 
IPA2= IPA.split(',')
Dic={'0':'ɑ̃','1':'ɔ̃','2':'ɛ̃','3':'œ̃','4':'tr','5':'kr','6':'dr','7':'gr'}
IPA_=[]
def ipa_(x):
  x=x.replace( 'ʁ','r') 
  x=x.replace('ɑ̃','0')
  x=x.replace('ɔ̃','1') 
  x=x.replace('ɛ̃', '2') 
  x=x.replace('œ̃','3') 
  x=x.replace('tr','4')
  x=x.replace('kr','5')
  x=x.replace('dr','6')
  x=x.replace('gr','7')
  x=x.replace( 'ʁ','r')
  return x 

df=df2

df['index_age']=df.index 
forjson=df[['index_age','mot']]


def splitage(x): 
    l=[]
    for y in x: 
        l.append(y)
    return l



def return_pho(x,choose=False):  
  l=[] 
  for i in ipa_(IPA): 
    for ch in x : 
      if i==x :
        l.append(i) 
  return l 



forjson['mot']=forjson['mot'].apply(lambda x: ipa_(str(x))) 

forjson['mot_ut']=forjson['mot'].apply(lambda x: splitage(str(x))) 


D={}
for i in forjson['index_age'].unique(): 
  D[i]=forjson[forjson['index_age']== i]  

index_json=forjson['index_age'].unique()

name_for_pho={}
for i in index_json : 
  l=D[i].mot.values
  k=list(D[i].mot_ut.values)
  name=[] 
  for p in range(len(k)) :
   for j in range(len(k[p])): 
    name.append(l[p])  
  name_for_pho[i]=name

 
final={} 
for i in index_json: 
  final[i]=pd.DataFrame([name_for_pho[i],D[i]['mot_ut'].sum()],index=['mot','phoneme']).transpose()


In [10]:
pho_child[['type','contenu']]

type           contenu
index   age                         
1_04_20 0      pho             mɔmɑ̃
        1      pho        kakatetəma
        2      pho              eːja
        3      pho  mmmnamna mnamnam
        4      pho                 u
...            ...               ...
5_07_06 18751  pho  e ty di zeʁo zɑ̃
        18752  pho                ta
        18753  pho                mə
        18754  pho                 m
        18755  pho                 a

[18756 rows x 2 columns]

In [0]:
def discrimi(x): 
  if x not in ipa_(IPA): 
    return np.nan 
  else : 
    return x 
for i in final.keys():
  final[i]['phoneme'] = final[i]['phoneme'].apply(lambda x: discrimi(x) ) 
  final[i].dropna(inplace=True)

In [0]:
fake_hour =(' '.join(list(final.keys()))).replace('_','/').split(' ') 
timer_add=0
for i in range(len(fake_hour)):
  fake_hour[i]='0'+''.join(list(fake_hour[i])[:-2])+'2010'+' '+time_fake

In [0]:
fichier = open("anae3.txt", "a")
itera=0
for key,df_val in final.items(): 
  for i in range(len(df_val)):
    to_write='{"date_time": "%s","text": "%s","value": "%s"}, \n' % (fake_hour[itera],df_val.mot.iloc[i],df_val.phoneme.iloc[i]) 
    fichier.write(to_write) 
itera+=1


fichier.close()



In [31]:
test_json=[]
for key,val in final.items():


,mot,phoneme
0,mɔm0,m
1,mɔm0,ɔ
2,mɔm0,m
3,mɔm0,0
4,kakatetəma,k
...,...,...
1390,tepapa,a
1391,tepapa,p
1392,tepapa,a
1393,n1,n


In [23]:
fake_hour

['1/04/2010 00:00:00',
 '1/06/2010 00:00:00',
 '1/07/2010 00:00:00',
 '1/09/2010 00:00:00',
 '1/10/2010 00:00:00',
 '2/00/2010 00:00:00',
 '2/00/2010 00:00:00',
 '2/01/2010 00:00:00',
 '2/02/2010 00:00:00',
 '2/03/2010 00:00:00',
 '2/04/2010 00:00:00',
 '2/05/2010 00:00:00',
 '2/06/2010 00:00:00',
 '2/08/2010 00:00:00',
 '2/09/2010 00:00:00',
 '2/10/2010 00:00:00',
 '2/11/2010 00:00:00',
 '3/01/2010 00:00:00',
 '3/03/2010 00:00:00',
 '3/04/2010 00:00:00',
 '3/05/2010 00:00:00',
 '3/06/2010 00:00:00',
 '3/08/2010 00:00:00',
 '3/10/2010 00:00:00',
 '4/00/2010 00:00:00',
 '4/01/2010 00:00:00',
 '4/04/2010 00:00:00',
 '4/05/2010 00:00:00',
 '4/08/2010 00:00:00',
 '5/01/2010 00:00:00',
 '5/07/2010 00:00:00']